# Build Your Own ROS 2 Robot from Scratch

## Goal 

Build your original ROS 2 arm robot 

## Steps 

1. Prepare the URDF 
2. Enable ROS 2 Control 
3. Run on ROS 2 apps
4. Run with the real robot

# 0. Who am I?

I'm Yutaka Kondo from Preferred Networks, Inc.

- GitHub: https://github.com/youtalk
- LinkedIn: https://www.linkedin.com/in/youtalk/
- Twitter: https://twitter.com/youtalk
- Company: https://www.preferred.jp/en/

In addition I'm an ex-organizer of ROS Japan Users Group and a program committee member of ROSCon JP.

- ROS Japan Users Group: https://rosjp.connpass.com
- ROSCon JP 2021: https://roscon.jp

I'm also an author of the world first ROS 2 book named ROS2ではじめよう次世代ロボットプログラミング (Getting Started Next Generation Robot Programming with ROS 2).

[![](https://www.youtalk.jp/get-started-ros2/img/cover.png)](https://www.amazon.co.jp/dp/B07W6DX9MW)

# 1. Prepare the URDF

This is the robot `pantilt_bot` we would like to build from scratch.

It has 2 degrees of freedom and a [Intel RealSense D435i](https://www.intelrealsense.com/depth-camera-d435i/) on top of the arm. 
[ROBOTIS Dynamixel motors XM430-W350](https://emanual.robotis.com/docs/en/dxl/x/xm430-w350/) are used for the joints. 

The CAD models of the motors and the hinge frames can be downloaded from http://en.robotis.com/service/downloadcenter.php

![](https://user-images.githubusercontent.com/579333/120786879-b3942300-c569-11eb-975c-e190e2b77b49.jpg)

## 1.1. Design your own arm robot

### Install Fusion 360

It's one of 3D CAD software. It has a personal license for hobby use and it's free.

https://www.autodesk.com/products/fusion-360/overview


### Design pantilt_bot

First import the motor STEP file and the hinge frame STEP files from http://en.robotis.com/service/downloadcenter.php

Then connect them using two rotational joints and one fixed joint.

Note that you must rename the root link to `base_link` due to the following exporter restriction.

![](https://user-images.githubusercontent.com/579333/120786885-b4c55000-c569-11eb-9796-71628a118db8.jpg)

## 1.2. Convert CAD file to URDF file

### Install URDF exporter for Fusion 360

`fusion2urdf` is very useful tool. It can convert a Fusion 360 model to the URDF model. 

https://github.com/syuntoku14/fusion2urdf

The  exporter generates the `pantilt_bot_description` package. Note that `fusion2urdf` can only export ROS 1 format.

### Video

[![](https://img.youtube.com/vi/aFr25ekHb04/0.jpg)](https://www.youtube.com/watch?v=aFr25ekHb04)

## 1.3. Migrate pantilt_bot_description to ROS 2

The exported `pantilt_bot_description` package is saved at `~/notebook_ws/pantilt_bot_description`.
You need to manually migrate it to ROS 2 format. 

```shell
$ cd ~/notebook_ws/pantilt_bot_description
$ ls -R
.:
CMakeLists.txt  launch  LICENSE  meshes  package.xml  urdf

./launch:
controller.launch  controller.yaml  display.launch  gazebo.launch  urdf.rviz

./meshes:
base_link.stl           joint2_horn_link_1.stl
joint1_horn_link_1.stl  joint2_link_1.stl

./urdf:
materials.xacro  pantilt_bot.gazebo  pantilt_bot.trans  pantilt_bot.xacro
```

We only use `meshes` directory, `urdf/pantilt_bot.xacro`, and `urdf/materials.xacro` without modification.
We need to migrate `package.xml` and `CMakeLists.txt`, and create launch files.

The ROS 2 formatted `pantilt_bot_description` package is located on `~/ros2_ws/src/dynamixel_contro/pantilt_bot_description`.

```shell
$ cd ~/ros2_ws/src/dynamixel_control/pantilt_bot_description/
$ ls -R
.:
CMakeLists.txt  controllers  launch  meshes  package.xml  urdf

./controllers:
controllers.yaml

./launch:
display.launch.py  pantilt_bot.launch.py  pantilt_bot.rviz

./meshes:
joint1.stl  joint1_horn.stl  joint2.stl  joint2_horn.stl

./urdf:
pantilt_bot.material.xacro      pantilt_bot.urdf.xacro
pantilt_bot.ros2_control.xacro  pantilt_bot.xacro
```

## 1.4. Attach Intel RealSense D435i sensor on top of the robot

Attach the Intel RealSense D435i sensor on top of the `pantilt_bot`. `realsense2_description` package contains the D435i URDF file. Because it is `xacro` format, you can easily integrate the D435i into `pantilt_bot`.

https://github.com/IntelRealSense/realsense-ros/tree/ros2/realsense2_description

![](https://user-images.githubusercontent.com/579333/120787232-1980aa80-c56a-11eb-9806-ac9123d361df.png)

The final version of the URDF file is located here: 
https://github.com/youtalk/dynamixel_control/blob/main/pantilt_bot_description/urdf/pantilt_bot.urdf.xacro

# 2. Enable ROS 2 Control

## 2.1. Set up ROSject for ROS 2 Foxy

When you launch this ROSject, run once the command below to completely enable ROS 2 Foxy environment variables.

```shell
$ sudo cp /home/user/ros2_ws/src/bash_aliases_backup /home/user/.bash_aliases
```

## 2.2. Build packages

First clone the `dynamixel_control` repogitory. It contains ROS 2 Control implementation for ROBOTIS Dynamixel motors.

https://github.com/youtalk/dynamixel_control

```shell
$ cd ~/ros2_ws/src
$ git clone https://github.com/youtalk/dynamixel_control.git -b rosdevday21
$ cd ~/ros2_ws
$ vcs import src < src/dynamixel_control/dynamixel_control.repos
```

Next install the dependencies and build them.

```shell
$ cd ~/ros2_ws
$ sudo apt update
$ rosdep update
$ rosdep install --from-paths src --ignore-src -r -y
$ colcon build --symlink-install
```

## 2.3. What is dynamixel_hardware package?

`dynamixel_hardware` package is the `SystemInterface` implementation of the `ros2_control` for the multiple ROBOTIS Dynamixel servos.

- https://ros-controls.github.io/control.ros.org/getting_started.html#architecture
- https://github.com/youtalk/dynamixel_control/tree/main/dynamixel_hardware

The `dynamixel_hardware` package is hopefully compatible any configuration of ROBOTIS Dynamixel servos thanks to the `ros2_control`'s flexible architecture.

### Configure ros2_control plugin

https://github.com/youtalk/dynamixel_control/blob/main/open_manipulator_x_description/urdf/open_manipulator_x.ros2_control.xacro#L9-L12

Update the `usb_port`, `baud_rate`, and `joint_ids` parameters on `urdf/pantilt_bot.ros2_control.xacro` to correctly communicate with Dynamixel servos.
The `use_dummy` parameter is required if you don't have a real `pantilt_bot`.

Note that `joint_ids` parameters must be splited by `,`.

```xml
<hardware>
  <plugin>dynamixel_hardware/DynamixelHardware</plugin>
  <param name="usb_port">/dev/ttyUSB0</param>
  <param name="baud_rate">1000000</param>
  <param name="joint_ids">11,12</param>
  <param name="use_dummy">true</param>
</hardware>
```

### Configure ros2_controllers plugin

https://github.com/youtalk/dynamixel_control/blob/main/pantilt_bot_description/controllers/controllers.yaml

You can use the `velocity_controller` for the velocity control and the `joint_trajectory_controller` for the position control with interpolation. The `joint_state_controller` publishes the robot's current joint state which contains the position, velocity and effort for each joints.

```yaml
controller_manager:
  ros__parameters:
    update_rate: 100  # Hz

    velocity_controller:
      type: velocity_controllers/JointGroupVelocityController

    joint_trajectory_controller:
      type: joint_trajectory_controller/JointTrajectoryController

    joint_state_controller:
      type: joint_state_controller/JointStateController

velocity_controller:
  ros__parameters:
    joints:
      - joint1
      - joint2

joint_trajectory_controller:
  ros__parameters:
    joints:
      - joint1
      - joint2
```

## 2.4. Check build

If you've successfully built them, the result of the following command shows the `RViz2` window and the `joint_state_publisher_gui` window.

```shell
$ . ~/ros2_ws/install/local_setup.bash
$ ros2 launch pantilt_bot_description display.launch.py
```

![](https://user-images.githubusercontent.com/579333/121196688-a6e63680-c8ab-11eb-8f73-e3d0ee8b9999.png)

# 3. Run on ROS 2 apps

## 3.1. ros2cli and RViz2

### Terminal 1

Launch the `ros2_control` controller and load the `follow_joint_trajectory_controller` and the `velocity_controller`.

```shell
$ . ~/ros2_ws/install/local_setup.bash
$ ros2 launch pantilt_bot_description pantilt_bot.launch.py
```

### Terminal 2

Switch the `pantilt_bot` controller to the `joint_trajectory_controller`.

```shell
$ . ~/ros2_ws/install/local_setup.bash
$ ros2 control switch_controllers --start joint_trajectory_controller
```

### Terminal 3

Send a `FollowJointTrajectory` action goal by ros2cli action interface.

```shell
$ . ~/ros2_ws/install/local_setup.bash
$ ros2 action send_goal /joint_trajectory_controller/follow_joint_trajectory control_msgs/action/FollowJointTrajectory -f "{
  trajectory: {
    joint_names: [joint1, joint2],
    points: [
      { positions: [0.0, 1.57], time_from_start: { sec: 1 } },
      { positions: [1.0, 1.57], time_from_start: { sec: 2 } },
      { positions: [-1.0, 1.57], time_from_start: { sec: 4 } },
      { positions: [-1.0, 1.0], time_from_start: { sec: 5 } },
      { positions: [1.0, 1.0], time_from_start: { sec: 7 } },
      { positions: [0.0, 1.0], time_from_start: { sec: 8 } },
      { positions: [0.0, 0.0], time_from_start: { sec: 9 } }
    ]
  }
}"
```

## 3.2. More applications

Once you have the `dynamixel_hardware` and the `pantilt_bot_description` packages, you can easily apply them to `MoveIt2` and `Gazebo` simulator.
Please try for your next step.

- https://github.com/ros-planning/moveit2
- https://github.com/ros-simulation/gazebo_ros2_control

# 4. Run with the real robot

## Set up on local

Follow the **2.2. Build packages** section on your local PC.

## Disable `use_dummpy`

Disable `use_dummy` parameter on the `pantilt_bot.ros2_control.xacro` to use the real `pantilt_bot`.

https://github.com/youtalk/dynamixel_control/blob/main/pantilt_bot_description/urdf/pantilt_bot.ros2_control.xacro

```diff
diff --git a/pantilt_bot_description/urdf/pantilt_bot.ros2_control.xacro b/pantilt_bot_description/urdf/pantilt_bot.ros2_control.xacro
index c6cdb74..111846d 100644
--- a/pantilt_bot_description/urdf/pantilt_bot.ros2_control.xacro
+++ b/pantilt_bot_description/urdf/pantilt_bot.ros2_control.xacro
@@ -9,7 +9,7 @@
         <param name="usb_port">/dev/ttyUSB0</param>
         <param name="baud_rate">1000000</param>
         <param name="joint_ids">11,12</param>
-        <param name="use_dummy">true</param>
+        <!-- <param name="use_dummy">true</param> -->
       </hardware>
       <joint name="joint1">
         <command_interface name="position"/>
```

## Demo

### Terminal 1

Launch the `ros2_control` controller and load the `follow_joint_trajectory_controller` and the `velocity_controller`.

```shell
$ . ~/ros2_ws/install/local_setup.bash
$ ros2 launch pantilt_bot_description pantilt_bot.launch.py
```

### Terminal 2

Switch the `pantilt_bot` controller to the `joint_trajectory_controller`.

```shell
$ . ~/ros2_ws/install/local_setup.bash
$ ros2 control switch_controllers --start joint_trajectory_controller
```

### Terminal 3

Send a `FollowJointTrajectory` action goal by ros2cli action interface.

```shell
$ . ~/ros2_ws/install/local_setup.bash
$ ros2 action send_goal /joint_trajectory_controller/follow_joint_trajectory control_msgs/action/FollowJointTrajectory -f "{
  trajectory: {
    joint_names: [joint1, joint2],
    points: [
      { positions: [0.0, 1.57], time_from_start: { sec: 1 } },
      { positions: [1.0, 1.57], time_from_start: { sec: 2 } },
      { positions: [-1.0, 1.57], time_from_start: { sec: 4 } },
      { positions: [-1.0, 1.0], time_from_start: { sec: 5 } },
      { positions: [1.0, 1.0], time_from_start: { sec: 7 } },
      { positions: [0.0, 1.0], time_from_start: { sec: 8 } },
      { positions: [0.0, 0.0], time_from_start: { sec: 9 } }
    ]
  }
}"
```

### Video

[![](https://img.youtube.com/vi/DzizMZp3KqU/0.jpg)](https://www.youtube.com/watch?v=DzizMZp3KqU)